# Initialization

In [ ]:
import itertools
import os
from pathlib import Path
from matplotlib import pyplot as plt
from pandas.core.base import DataError
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from notebooks.heart_transplant.dependencies.heart_transplant_functions import get_rolling_cv, \
    remove_missing_columns, format_feature

for folder in itertools.chain([Path.cwd()], Path.cwd().parents):
    if (folder / 'Pipfile').exists():
        os.chdir(folder)
        break

from notebooks.heart_transplant.dependencies.heart_transplant_data import *

%load_ext autoreload
%autoreload 2
%load_ext autotime
%config InlineBackend.figure_format = 'retina'

plt.style.use('dark_background')

X = X_reduced[continuous_features]
y = y_reduced

top_40_rf_features = ['log_ischtime', 'log_tbili', 'log_creat_trr', 'hist_mi', 'log_cmassratio', 'weight ratio', 'congenital', 'ecd_donor', 'log_wgt_kg_calc', 'log_sgpt_don', 'log_sgot_don', 'log_age_don', 'log_wgt_kg_don_calc', 'log_distance', 'log_age', 'cdc_risk_hiv_don', 'log_hgt_cm_calc', 'log_hgt_cm_don_calc', 'log_bun_don', 'coronary_angio', 'log_tbili_don', 'tattoos', 'biopsy_dgn', 'log_creat_don', 'vessels_50sten', 'log_most_rcnt_creat', 'lv_eject', 'education', 'retransplant', 'log_newpra', 'protein_urine', 'hematocrit_don', 'iabp_tcr', 'hist_iv_drug_old_don', 'hiv_serostatus', 'gender', 'cmv_status', 'hemo_co_tcr', 'impl_defibril', 'hist_hypertens_don']

In [ ]:
scaler = use_df(StandardScaler)().fit(X)
X_scaled  = scaler.transform(X)

for feature in top_40_rf_features:
    if feature not in X.columns:
        print('Skipping', feature)
        continue

    try:
        plt.title(format_feature(metadata, feature))
        X_scaled[y==1][feature].groupby(X_scaled.assign(tx_year=dataset_raw['tx_year'])['tx_year']).mean().plot(label='Died')
        X_scaled[y==0][feature].groupby(X_scaled.assign(tx_year=dataset_raw['tx_year'])['tx_year']).mean().plot(label='Survived')
        # X.assign(tx_year=dataset_raw['tx_year']).plot(x='tx_year', y=feature, kind='scatter', c=['red' if y else 'blue' for y in y.values], alpha=0.2)
        plt.legend()
        plt.ylim(-0.5, 0.5)
    except DataError:
        pass
    else:
        plt.show()